In [1]:
import pandas as pd
import numpy as np
import itertools 
from sklearn.model_selection import train_test_split
from sklearn import linear_model, svm
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix,accuracy_score, recall_score, roc_auc_score, auc, f1_score
import matplotlib.pyplot as plt
import datetime
from sklearn.kernel_approximation import RBFSampler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import plot_tree
from xgboost import plot_importance
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import category_encoders as ce

from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV

C:\Users\RAUL\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\RAUL\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.options.display.max_columns = 999

In [4]:
file = 'C:\\Users\\RAUL\\Documents\\OrgDatos2\\Data\\events_TP2.csv'
testFile = 'C:\\Users\\RAUL\\Documents\\OrgDatos2\\Data\\events_TEST.csv'
trainingFile = 'C:\\Users\\RAUL\\Documents\\OrgDatos2\\Data\\events_TRAINING.csv'

In [5]:
df = pd.read_csv(file)

In [6]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [7]:
df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.sort_values(['person', 'timestamp'] , inplace = True)

In [9]:
df['timestamp'].max() - datetime.timedelta(days=15)

Timestamp('2018-05-16 23:59:59')

In [10]:
personas = df[['person']].drop_duplicates()

In [11]:
def sesiones_feature(df, personas, tiempo_inactividad, dias): #esta funcion analiza las sesiones de los usuarios considerando "sesiones" al conjunto de actividades seguidas sin que pase un tiempo determinado como tiempo de inactividad
    df = df[df['timestamp'] >= (df['timestamp'].max() - datetime.timedelta(days=dias))]
    gt_30min = df.timestamp.diff() > datetime.timedelta(minutes=tiempo_inactividad)
    diff_user = df.person != df.person.shift()
    df['# Visita'] = (diff_user | gt_30min).cumsum()
    
    cantidad_sesiones_col = 'cant_sesiones_inac' + str(tiempo_inactividad) + 'MIN_ult_' + str(dias) + '_dias' 
    promedio_sesiones_sec_col = 'promedio_sesiones_sec_inac' + str(tiempo_inactividad) + 'MIN_ult_' + str(dias) + '_dias' 
    tiempo_total_sesiones_sec_col = 'tiempo_total_sesiones_sec_inac' + str(tiempo_inactividad) + 'MIN_ult_' + str(dias) + '_dias' 
    
    duracion_sesiones = pd.DataFrame(df.groupby(['person', '# Visita'])['timestamp'].agg(lambda x : x.max() - x.min())).reset_index()
    sumas_sesiones = pd.DataFrame(duracion_sesiones.groupby('person')['timestamp'].sum()).reset_index().rename(columns = {'timestamp' : 'sum'})
    cantidad_sesiones =  pd.DataFrame(duracion_sesiones.groupby('person')['timestamp'].count()).reset_index().rename(columns = {'timestamp' : cantidad_sesiones_col})
    sesiones = sumas_sesiones.merge(cantidad_sesiones, on = 'person')   
    sesiones['promedio_sesiones'] = pd.to_timedelta(pd.to_timedelta(sesiones['sum'])/sesiones[cantidad_sesiones_col], unit = 'ns').dt.round('1s')
    sesiones[promedio_sesiones_sec_col] = sesiones['promedio_sesiones'].dt.total_seconds()
    sesiones[tiempo_total_sesiones_sec_col] = sesiones['sum'].dt.total_seconds()
    del sesiones['promedio_sesiones']
    del sesiones['sum']
    return personas.merge(sesiones, on = 'person', how = 'left').fillna(0)

In [12]:
#act30 = df[df['timestamp'] >= pd.to_datetime('2018-04-30')].groupby('person')['timestamp'].count().reset_index().rename(columns = {'timestamp' : '#act_ult30dias'})

In [13]:
def marcas_vistas_feature(df, personas): #Esta funcion analiza las marcas más vistas por cada usuario
    samsung_vistos = df[(df['event'] == 'viewed product') & (df['model'].str.lower().str.contains('samsung') == True)].groupby('person')['timestamp'].count().reset_index().rename(columns = {'timestamp' : 'samsung_vistos'})
    iphone_vistos  = df[(df['event'] == 'viewed product') & (df['model'].str.lower().str.contains('iphone') == True)].groupby('person')['timestamp'].count().reset_index().rename(columns = {'timestamp' : 'iphone_vistos'})
    motorola_vistos = df[(df['event'] == 'viewed product') & (df['model'].str.lower().str.contains('motorola') == True)].groupby('person')['timestamp'].count().reset_index().rename(columns = {'timestamp' : 'motorola_vistos'})
    marcas_vistas = samsung_vistos.merge(iphone_vistos, on = 'person', how = 'left').fillna(0)
    marcas_vistas = marcas_vistas.merge(motorola_vistos, on = 'person', how = 'left').fillna(0)
    marcas_vistas['total_marcas_vistas'] = marcas_vistas['samsung_vistos'] + marcas_vistas['iphone_vistos'] + marcas_vistas['motorola_vistos']
    marcas_vistas['%_vistos_samsung'] = marcas_vistas['samsung_vistos'] / marcas_vistas['total_marcas_vistas']
    marcas_vistas['%_vistos_motorola'] = marcas_vistas['motorola_vistos'] / marcas_vistas['total_marcas_vistas']
    marcas_vistas['%_vistos_iphone'] = marcas_vistas['iphone_vistos'] / marcas_vistas['total_marcas_vistas']
    personas = personas.merge(marcas_vistas, on = 'person', how = 'left')
    personas['total_marcas_vistas'] = personas['total_marcas_vistas'].fillna(0)
    personas['%_vistos_samsung'] = personas['%_vistos_samsung'].fillna(0)
    personas['%_vistos_iphone'] = personas['%_vistos_iphone'].fillna(0)
    personas['%_vistos_motorola'] = personas['%_vistos_motorola'].fillna(0)
    personas['samsung_vistos'] = personas['samsung_vistos'].fillna(0)
    personas['iphone_vistos'] = personas['iphone_vistos'].fillna(0)
    personas['motorola_vistos'] = personas['motorola_vistos'].fillna(0)
    return personas

In [14]:
def modelos_vistos_feature(df, personas): #Esta funcion analiza los modelos vistos por las personas
    modelos_vistos = df[df['event'] == 'viewed product'].groupby('person')['model'].agg({'modelos_vistos': 'count','modelos_distintos': 'nunique'})
    sku_vistos = df[df['event'] == 'viewed product'].groupby('person')['sku'].agg({'sku_distintos': 'nunique'})
    vistos = modelos_vistos.merge(sku_vistos, on  = 'person')
    personas = personas.merge(vistos, on = 'person', how = 'left')
    personas['modelos_vistos'] = personas['modelos_vistos'].fillna(0)
    personas['modelos_distintos'] = personas['modelos_distintos'].fillna(0)
    personas['sku_distintos'] = personas['sku_distintos'].fillna(0)
    return personas

Analizaremos los caminos que realizan las personas antes de comprar

In [15]:
def modelos_vistos_ult_x_dias(df, personas, dias): #Analiza los modelos vistos en los x dias atras de la ultima fecha del df
    df = df[df['timestamp'] >= (df['timestamp'].max() - datetime.timedelta(days=dias))]
    
    mas_visto_col = 'mod_mas_visto_ult_' + str(dias) + '_dias'
    total_vistos_col = 'total_vistos_ult_' + str(dias) + '_dias'
    por_mas_vistos_col = '%_mas_visto_ult_' + str(dias) + '_dias' 
    
    modelos_mas_vistos = pd.DataFrame(df[df['event'] == 'viewed product'].groupby('person')['model'].agg(lambda x:x.value_counts()[0])).reset_index().rename(columns = {'model' : mas_visto_col})
    total_vistos = pd.DataFrame(df[df['event'] == 'viewed product'].groupby('person')['model'].count().reset_index().rename(columns = {'model' : total_vistos_col}))

    modelos_mas_vistos = modelos_mas_vistos.merge(total_vistos, on = 'person')
    modelos_mas_vistos[por_mas_vistos_col] = modelos_mas_vistos[mas_visto_col]/modelos_mas_vistos[total_vistos_col]
    
    personas = personas.merge(modelos_mas_vistos, how = 'left', on = 'person')
    
    personas[por_mas_vistos_col] = personas[por_mas_vistos_col].fillna(0)
    personas[mas_visto_col] = personas[mas_visto_col].fillna(0)
    personas[total_vistos_col] = personas[total_vistos_col].fillna(0)
    
    return personas

In [16]:
def compro(x):
    if('conversion' in x):
        return True
    else:
        return False
    
def borrarRepe(lista):
    return [x[0] for x in itertools.groupby(lista)]

def eliminarConv(x):
    if('conversion' in x):
        return x.remove('conversion')
    else:
        return x

In [17]:
maxdate = max(df['timestamp'])

In [18]:
maxdate.date()

datetime.date(2018, 5, 31)

In [19]:
def caminos_feature(df, personas): #esta funcion analiza los "caminos" que llevan a la compra de los usuarios
    ultima_fecha = pd.to_datetime(max(df['timestamp']))
    paths = df.sort_values(by = 'timestamp').copy()
    paths['#Camino'] = paths.groupby('person')['event'].apply(lambda x : x.eq('conversion').shift().fillna(0).cumsum())
    #Veremos la cantidad de modelos y Sku vistos en cada camino
    paths2 = paths.groupby(['person','#Camino'])['event'].apply(list).reset_index().rename(columns = {'event' : 'camino'})
    camino = paths.groupby(['person','#Camino'])['event'].apply(list).reset_index().rename(columns = {'event' : 'camino'})['camino']
    paths2['compro'] = paths2['camino'].apply(lambda x : compro(x))
    paths2.rename(columns = {'camino' : 'camino_sin_compra'}, inplace = True)
    paths2['camino_con_compra'] = camino.copy()
    paths2['#pasos_antes_comprar'] = paths2['camino_sin_compra'].apply(lambda x : len(x))
    camino2 = paths2['camino_sin_compra'].copy()
    paths2['camino_sin_compra_sin_rep'] = camino2.apply(lambda x : borrarRepe(x))
    paths2 = paths2.merge(paths.groupby(['person', '#Camino'])['timestamp'].agg({'primer_paso_time' : 'min', 'ultimo_paso_time' : 'max'}), on = ['person','#Camino'])
    paths2['primer_paso_date'] = pd.to_datetime(paths2['primer_paso_time'].dt.date)
    paths2['ultimo_paso_date'] = pd.to_datetime(paths2['ultimo_paso_time'].dt.date)
    paths2['primera_actividad_ultimo_paso'] = (ultima_fecha - paths2['primer_paso_date']).dt.days
    paths2['ultima_actividad'] = (ultima_fecha - paths2['ultimo_paso_date']).dt.days
    del paths2['primer_paso_time']
    del paths2['ultimo_paso_time']
    del paths2['camino_sin_compra']
    del paths2['camino_con_compra']
    del paths2['camino_sin_compra_sin_rep']
    del paths2['primer_paso_date']
    del paths2['ultimo_paso_date']
    personas =  personas.merge(paths2, on = 'person', how = 'left')
    personas.drop_duplicates(subset = 'person', keep = 'last', inplace = True)
    return personas

In [20]:
def primera_actividad_feature(df, personas):#Analiza en qué fecha se realizo la primera actividad
    ultima_fecha = pd.to_datetime(max(df['timestamp']))
    primera_actividad = df.groupby('person')['timestamp'].min().reset_index().rename(columns = {'timestamp' : 'fecha_primera_actividad'})
    primera_actividad['fecha_primera_actividad'] = pd.to_datetime(primera_actividad['fecha_primera_actividad'].dt.date)
    primera_actividad['primera_actividad'] = (ultima_fecha - primera_actividad['fecha_primera_actividad']).dt.days
    del primera_actividad['fecha_primera_actividad']
    return personas.merge(primera_actividad, on = 'person', how ='left')

In [21]:
def cantidad_de_actividades_feature(df, personas):#Analiza la cantidad de actividades de cada persona
    actividades = df.groupby('person')['timestamp'].count().reset_index().rename(columns = {'timestamp' : '# Actividades'})
    return personas.merge(actividades, on = 'person', how ='left')

In [22]:
def actividad_ult_x_dias(df, personas, dias):# Analiza la cantidad de actividades en los ultimos X dias
    col_name = 'act_ult_' + str(dias) + '_dias'
    act_dias = pd.DataFrame(df[df['timestamp'] >= (df['timestamp'].max() - datetime.timedelta(days=dias))].groupby('person')['timestamp'].count()).reset_index().rename(columns = {'timestamp' : col_name})
    personas = personas.merge(act_dias, on = 'person', how = 'left') 
    personas[col_name] = personas[col_name].fillna(0)
    
    return personas

In [23]:
def checkout_ult_x_dias(df, personas, dias):# Analiza la cantidad de checkouts en los ultimos X dias
    col_name = 'check_ult_' + str(dias) + '_dias'
    chk_dias = pd.DataFrame(df[(df['timestamp'] >= (df['timestamp'].max() - datetime.timedelta(days=dias))) & (df['event'] == 'checkout')].groupby('person')['timestamp'].count()).reset_index().rename(columns = {'timestamp' : col_name})
    personas = personas.merge(chk_dias, on = 'person', how = 'left') 
    personas[col_name] = personas[col_name].fillna(0)
    
    return personas

In [24]:
def ultimo_checkout(df, personas):#Analiza cuando realizo el ultimo checkout
    ultima_fecha = df['timestamp'].max()
    ult_check = df[df['event'] == 'checkout'].groupby('person')['timestamp'].max().reset_index().rename(columns = {'timestamp' : 'ult_checkout'}) 
    ult_check['ult_checkout_dias']  = (ultima_fecha - ult_check['ult_checkout']).dt.days
    ult_check['ult_checkout_segundos']  = (ultima_fecha - ult_check['ult_checkout']).dt.total_seconds()
    del ult_check['ult_checkout']
    
    return personas.merge(ult_check, on = 'person', how = 'left')

In [25]:
def mod_dist_check_ult_x_dias(df, personas, dias):#Analiza la cantidad de modelos distintos que puso en checkout en los ultimos X días
    col_name = 'mod_dist_check_ult_' + str(dias) + '_dias'
    mod_dias = pd.DataFrame(df[(df['timestamp'] >= (df['timestamp'].max() - datetime.timedelta(days=dias))) & (df['event'] == 'checkout')].groupby('person')['model'].nunique()).reset_index().rename(columns = {'model' : col_name})
    personas = personas.merge(mod_dias, on = 'person', how = 'left') 
    personas[col_name] = personas[col_name].fillna(0)
    return personas


In [26]:
def conv_ult_x_dias(df, personas, dias):#Analiza la cantidad de conversiones en los ultimos X dias
    col_name = 'conv_ult_' + str(dias) + '_dias'
    act_dias = pd.DataFrame(df[(df['timestamp'] >= (df['timestamp'].max() - datetime.timedelta(days=dias))) & (df['event'] == 'conversion')].groupby('person')['timestamp'].count()).reset_index().rename(columns = {'timestamp' : col_name})
    personas = personas.merge(act_dias, on = 'person', how = 'left') 
    personas[col_name] = personas[col_name].fillna(0)
    
    return personas

In [27]:
def info_demografica(df, personas):    
    paises = df.groupby('person')['country'].first().reset_index()
    regiones = df.groupby('person')['region'].first().reset_index()
    ciudades = df.groupby('person')['city'].first().reset_index()

    demografica = paises.merge(regiones, on = 'person')
    demografica = demografica.merge(ciudades, on = 'person')

    encoder = ce.BinaryEncoder(cols=['country', 'region', 'city'])
    demografica_binary = encoder.fit_transform(demografica)

    return personas.merge(demografica_binary, on = 'person', how = 'left')

In [28]:
personas = sesiones_feature(df, personas, 45, 9999)
personas = sesiones_feature(df, personas, 45, 30)
personas = sesiones_feature(df, personas, 45, 25)
personas = sesiones_feature(df, personas, 45, 20)
personas = sesiones_feature(df, personas, 45, 15)
personas = sesiones_feature(df, personas, 45, 10)
personas = sesiones_feature(df, personas, 45, 5)
personas = sesiones_feature(df, personas, 45, 1)

In [29]:
personas = sesiones_feature(df, personas, 30, 9999)
personas = sesiones_feature(df, personas, 30, 30)
personas = sesiones_feature(df, personas, 30, 25)
personas = sesiones_feature(df, personas, 30, 20)
personas = sesiones_feature(df, personas, 30, 15)
personas = sesiones_feature(df, personas, 30, 10)
personas = sesiones_feature(df, personas, 30, 5)
personas = sesiones_feature(df, personas, 30, 1)

In [30]:
personas = marcas_vistas_feature(df, personas)

In [31]:
personas.head()

,person,cant_sesiones_inac45MIN_ult_9999_dias,promedio_sesiones_sec_inac45MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac45MIN_ult_9999_dias,cant_sesiones_inac45MIN_ult_30_dias,promedio_sesiones_sec_inac45MIN_ult_30_dias,tiempo_total_sesiones_sec_inac45MIN_ult_30_dias,cant_sesiones_inac45MIN_ult_25_dias,promedio_sesiones_sec_inac45MIN_ult_25_dias,tiempo_total_sesiones_sec_inac45MIN_ult_25_dias,cant_sesiones_inac45MIN_ult_20_dias,promedio_sesiones_sec_inac45MIN_ult_20_dias,tiempo_total_sesiones_sec_inac45MIN_ult_20_dias,cant_sesiones_inac45MIN_ult_15_dias,promedio_sesiones_sec_inac45MIN_ult_15_dias,tiempo_total_sesiones_sec_inac45MIN_ult_15_dias,cant_sesiones_inac45MIN_ult_10_dias,promedio_sesiones_sec_inac45MIN_ult_10_dias,tiempo_total_sesiones_sec_inac45MIN_ult_10_dias,cant_sesiones_inac45MIN_ult_5_dias,promedio_sesiones_sec_inac45MIN_ult_5_dias,tiempo_total_sesiones_sec_inac45MIN_ult_5_dias,cant_sesiones_inac45MIN_ult_1_dias,promedio_sesiones_sec_inac45MIN_ult_1_dias,tiempo_total_sesiones_sec_inac45MIN_ult_1_dias,cant_sesiones_inac30MIN_ult_9999_dias,promedio_sesiones_sec_inac30MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac30MIN_ult_9999_dias,cant_sesiones_inac30MIN_ult_30_dias,promedio_sesiones_sec_inac30MIN_ult_30_dias,tiempo_total_sesiones_sec_inac30MIN_ult_30_dias,cant_sesiones_inac30MIN_ult_25_dias,promedio_sesiones_sec_inac30MIN_ult_25_dias,tiempo_total_sesiones_sec_inac30MIN_ult_25_dias,cant_sesiones_inac30MIN_ult_20_dias,promedio_sesiones_sec_inac30MIN_ult_20_dias,tiempo_total_sesiones_sec_inac30MIN_ult_20_dias,cant_sesiones_inac30MIN_ult_15_dias,promedio_sesiones_sec_inac30MIN_ult_15_dias,tiempo_total_sesiones_sec_inac30MIN_ult_15_dias,cant_sesiones_inac30MIN_ult_10_dias,promedio_sesiones_sec_inac30MIN_ult_10_dias,tiempo_total_sesiones_sec_inac30MIN_ult_10_dias,cant_sesiones_inac30MIN_ult_5_dias,promedio_sesiones_sec_inac30MIN_ult_5_dias,tiempo_total_sesiones_sec_inac30MIN_ult_5_dias,cant_sesiones_inac30MIN_ult_1_dias,promedio_sesiones_sec_inac30MIN_ult_1_dias,tiempo_total_sesiones_sec_inac30MIN_ult_1_dias,samsung_vistos,iphone_vistos,motorola_vistos,total_marcas_vistas,%_vistos_samsung,%_vistos_motorola,%_vistos_iphone
0,0008ed71,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,00091926,31,708.0,21934.0,31.0,708.0,21934.0,27.0,761.0,20544.0,22.0,756.0,16625.0,14.0,830.0,11623.0,9.0,396.0,3562.0,5.0,244.0,1222.0,2.0,124.0,248.0,34,438.0,14890.0,34.0,438.0,14890.0,30.0,450.0,13500.0,24.0,505.0,12116.0,15.0,610.0,9149.0,9.0,396.0,3562.0,5.0,244.0,1222.0,2.0,124.0,248.0,61.0,251.0,55.0,367.0,0.166213,0.149864,0.683924
2,00091a7a,1,347.0,347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,347.0,347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,000ba417,5,2541.0,12703.0,5.0,2541.0,12703.0,5.0,2541.0,12703.0,5.0,2541.0,12703.0,5.0,2541.0,12703.0,2.0,4813.0,9626.0,0.0,0.0,0.0,0.0,0.0,0.0,5,2541.0,12703.0,5.0,2541.0,12703.0,5.0,2541.0,12703.0,5.0,2541.0,12703.0,5.0,2541.0,12703.0,2.0,4813.0,9626.0,0.0,0.0,0.0,0.0,0.0,0.0,105.0,8.0,35.0,148.0,0.709459,0.236486,0.054054
4,000c79fe,1,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,0.0,0.0,0.0,1,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [32]:
personas = modelos_vistos_feature(df, personas)

In [33]:
personas.head()

,person,cant_sesiones_inac45MIN_ult_9999_dias,promedio_sesiones_sec_inac45MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac45MIN_ult_9999_dias,cant_sesiones_inac45MIN_ult_30_dias,promedio_sesiones_sec_inac45MIN_ult_30_dias,tiempo_total_sesiones_sec_inac45MIN_ult_30_dias,cant_sesiones_inac45MIN_ult_25_dias,promedio_sesiones_sec_inac45MIN_ult_25_dias,tiempo_total_sesiones_sec_inac45MIN_ult_25_dias,cant_sesiones_inac45MIN_ult_20_dias,promedio_sesiones_sec_inac45MIN_ult_20_dias,tiempo_total_sesiones_sec_inac45MIN_ult_20_dias,cant_sesiones_inac45MIN_ult_15_dias,promedio_sesiones_sec_inac45MIN_ult_15_dias,tiempo_total_sesiones_sec_inac45MIN_ult_15_dias,cant_sesiones_inac45MIN_ult_10_dias,promedio_sesiones_sec_inac45MIN_ult_10_dias,tiempo_total_sesiones_sec_inac45MIN_ult_10_dias,cant_sesiones_inac45MIN_ult_5_dias,promedio_sesiones_sec_inac45MIN_ult_5_dias,tiempo_total_sesiones_sec_inac45MIN_ult_5_dias,cant_sesiones_inac45MIN_ult_1_dias,promedio_sesiones_sec_inac45MIN_ult_1_dias,tiempo_total_sesiones_sec_inac45MIN_ult_1_dias,cant_sesiones_inac30MIN_ult_9999_dias,promedio_sesiones_sec_inac30MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac30MIN_ult_9999_dias,cant_sesiones_inac30MIN_ult_30_dias,promedio_sesiones_sec_inac30MIN_ult_30_dias,tiempo_total_sesiones_sec_inac30MIN_ult_30_dias,cant_sesiones_inac30MIN_ult_25_dias,promedio_sesiones_sec_inac30MIN_ult_25_dias,tiempo_total_sesiones_sec_inac30MIN_ult_25_dias,cant_sesiones_inac30MIN_ult_20_dias,promedio_sesiones_sec_inac30MIN_ult_20_dias,tiempo_total_sesiones_sec_inac30MIN_ult_20_dias,cant_sesiones_inac30MIN_ult_15_dias,promedio_sesiones_sec_inac30MIN_ult_15_dias,tiempo_total_sesiones_sec_inac30MIN_ult_15_dias,cant_sesiones_inac30MIN_ult_10_dias,promedio_sesiones_sec_inac30MIN_ult_10_dias,tiempo_total_sesiones_sec_inac30MIN_ult_10_dias,cant_sesiones_inac30MIN_ult_5_dias,promedio_sesiones_sec_inac30MIN_ult_5_dias,tiempo_total_sesiones_sec_inac30MIN_ult_5_dias,cant_sesiones_inac30MIN_ult_1_dias,promedio_sesiones_sec_inac30MIN_ult_1_dias,tiempo_total_sesiones_sec_inac30MIN_ult_1_dias,samsung_vistos,iphone_vistos,motorola_vistos,total_marcas_vistas,%_vistos_samsung,%_vistos_motorola,%_vistos_iphone,modelos_vistos,modelos_distintos,sku_distintos
0,0008ed71,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,00091926,31,708.0,21934.0,31.0,708.0,21934.0,27.0,761.0,20544.0,22.0,756.0,16625.0,14.0,830.0,11623.0,9.0,396.0,3562.0,5.0,244.0,1222.0,2.0,124.0,248.0,34,438.0,14890.0,34.0,438.0,14890.0,30.0,450.0,13500.0,24.0,505.0,12116.0,15.0,610.0,9149.0,9.0,396.0,3562.0,5.0,244.0,1222.0,2.0,124.0,248.0,61.0,251.0,55.0,367.0,0.166213,0.149864,0.683924,372.0,36.0,164.0
2,00091a7a,1,347.0,347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,347.0,347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,3.0,3.0,3.0
3,000ba417,5,2541.0,12703.0,5.0,2541.0,12703.0,5.0,2541.0,12703.0,5.0,2541.0,12703.0,5.0,2541.0,12703.0,2.0,4813.0,9626.0,0.0,0.0,0.0,0.0,0.0,0.0,5,2541.0,12703.0,5.0,2541.0,12703.0,5.0,2541.0,12703.0,5.0,2541.0,12703.0,5.0,2541.0,12703.0,2.0,4813.0,9626.0,0.0,0.0,0.0,0.0,0.0,0.0,105.0,8.0,35.0,148.0,0.709459,0.236486,0.054054,153.0,26.0,81.0
4,000c79fe,1,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,0.0,0.0,0.0,1,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,1.0,620.0,620.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,3.0,1.0,1.0


In [34]:
personas = modelos_vistos_ult_x_dias(df, personas, 9999)
personas = modelos_vistos_ult_x_dias(df, personas, 30)
personas = modelos_vistos_ult_x_dias(df, personas, 25)
personas = modelos_vistos_ult_x_dias(df, personas, 20)
personas = modelos_vistos_ult_x_dias(df, personas, 15)
personas = modelos_vistos_ult_x_dias(df, personas, 10)
personas = modelos_vistos_ult_x_dias(df, personas, 5)
personas = modelos_vistos_ult_x_dias(df, personas, 1)

In [35]:
personas.sample(10)

,person,cant_sesiones_inac45MIN_ult_9999_dias,promedio_sesiones_sec_inac45MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac45MIN_ult_9999_dias,cant_sesiones_inac45MIN_ult_30_dias,promedio_sesiones_sec_inac45MIN_ult_30_dias,tiempo_total_sesiones_sec_inac45MIN_ult_30_dias,cant_sesiones_inac45MIN_ult_25_dias,promedio_sesiones_sec_inac45MIN_ult_25_dias,tiempo_total_sesiones_sec_inac45MIN_ult_25_dias,cant_sesiones_inac45MIN_ult_20_dias,promedio_sesiones_sec_inac45MIN_ult_20_dias,tiempo_total_sesiones_sec_inac45MIN_ult_20_dias,cant_sesiones_inac45MIN_ult_15_dias,promedio_sesiones_sec_inac45MIN_ult_15_dias,tiempo_total_sesiones_sec_inac45MIN_ult_15_dias,cant_sesiones_inac45MIN_ult_10_dias,promedio_sesiones_sec_inac45MIN_ult_10_dias,tiempo_total_sesiones_sec_inac45MIN_ult_10_dias,cant_sesiones_inac45MIN_ult_5_dias,promedio_sesiones_sec_inac45MIN_ult_5_dias,tiempo_total_sesiones_sec_inac45MIN_ult_5_dias,cant_sesiones_inac45MIN_ult_1_dias,promedio_sesiones_sec_inac45MIN_ult_1_dias,tiempo_total_sesiones_sec_inac45MIN_ult_1_dias,cant_sesiones_inac30MIN_ult_9999_dias,promedio_sesiones_sec_inac30MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac30MIN_ult_9999_dias,cant_sesiones_inac30MIN_ult_30_dias,promedio_sesiones_sec_inac30MIN_ult_30_dias,tiempo_total_sesiones_sec_inac30MIN_ult_30_dias,cant_sesiones_inac30MIN_ult_25_dias,promedio_sesiones_sec_inac30MIN_ult_25_dias,tiempo_total_sesiones_sec_inac30MIN_ult_25_dias,cant_sesiones_inac30MIN_ult_20_dias,promedio_sesiones_sec_inac30MIN_ult_20_dias,tiempo_total_sesiones_sec_inac30MIN_ult_20_dias,cant_sesiones_inac30MIN_ult_15_dias,promedio_sesiones_sec_inac30MIN_ult_15_dias,tiempo_total_sesiones_sec_inac30MIN_ult_15_dias,cant_sesiones_inac30MIN_ult_10_dias,promedio_sesiones_sec_inac30MIN_ult_10_dias,tiempo_total_sesiones_sec_inac30MIN_ult_10_dias,cant_sesiones_inac30MIN_ult_5_dias,promedio_sesiones_sec_inac30MIN_ult_5_dias,tiempo_total_sesiones_sec_inac30MIN_ult_5_dias,cant_sesiones_inac30MIN_ult_1_dias,promedio_sesiones_sec_inac30MIN_ult_1_dias,tiempo_total_sesiones_sec_inac30MIN_ult_1_dias,samsung_vistos,iphone_vistos,motorola_vistos,total_marcas_vistas,%_vistos_samsung,%_vistos_motorola,%_vistos_iphone,modelos_vistos,modelos_distintos,sku_distintos,mod_mas_visto_ult_9999_dias,total_vistos_ult_9999_dias,%_mas_visto_ult_9999_dias,mod_mas_visto_ult_30_dias,total_vistos_ult_30_dias,%_mas_visto_ult_30_dias,mod_mas_visto_ult_25_dias,total_vistos_ult_25_dias,%_mas_visto_ult_25_dias,mod_mas_visto_ult_20_dias,total_vistos_ult_20_dias,%_mas_visto_ult_20_dias,mod_mas_visto_ult_15_dias,total_vistos_ult_15_dias,%_mas_visto_ult_15_dias,mod_mas_visto_ult_10_dias,total_vistos_ult_10_dias,%_mas_visto_ult_10_dias,mod_mas_visto_ult_5_dias,total_vistos_ult_5_dias,%_mas_visto_ult_5_dias,mod_mas_visto_ult_1_dias,total_vistos_ult_1_dias,%_mas_visto_ult_1_dias
35254,e84bb55e,1,980.0,980.0,1.0,980.0,980.0,1.0,980.0,980.0,1.0,980.0,980.0,1.0,980.0,980.0,1.0,980.0,980.0,1.0,980.0,980.0,1.0,980.0,980.0,1,980.0,980.0,1.0,980.0,980.0,1.0,980.0,980.0,1.0,980.0,980.0,1.0,980.0,980.0,1.0,980.0,980.0,1.0,980.0,980.0,1.0,980.0,980.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,15.0,3.0,11.0,10.0,15.0,0.666667,10.0,15.0,0.666667,10.0,15.0,0.666667,10.0,15.0,0.666667,10.0,15.0,0.666667,10.0,15.0,0.666667,10.0,15.0,0.666667,10.0,15.0,0.666667
23324,9a886433,3,63.0,189.0,3.0,63.0,189.0,2.0,88.0,176.0,1.0,176.0,176.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,63.0,189.0,3.0,63.0,189.0,2.0,88.0,176.0,1.0,176.0,176.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,0.250000,0.000000,0.750000,4.0,3.0,4.0,2.0,4.0,0.500000,2.0,4.0,0.500000,2.0,3.0,0.666667,2.0,2.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
16228,6af9b6e7,1,390.0,390.0,1.0,390.0,390.0,1.0,390.0,390.0,1.0,390.0,390.0,1.0,390.0,390.0,1.0,390.0,390.0,0.0,0.0,0.0,0.0,0.0,0.0,1,390.0,390.0,1.0,390.0,390.0,1.0,390.0,390.0,1.0,390.0,390.0,1.0,390.0,390.0,1.0,390.0,390.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,5.0,0.800000,0.000000,

In [36]:
personas = caminos_feature(df, personas)

In [37]:
personas.head()

,person,cant_sesiones_inac45MIN_ult_9999_dias,promedio_sesiones_sec_inac45MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac45MIN_ult_9999_dias,cant_sesiones_inac45MIN_ult_30_dias,promedio_sesiones_sec_inac45MIN_ult_30_dias,tiempo_total_sesiones_sec_inac45MIN_ult_30_dias,cant_sesiones_inac45MIN_ult_25_dias,promedio_sesiones_sec_inac45MIN_ult_25_dias,tiempo_total_sesiones_sec_inac45MIN_ult_25_dias,cant_sesiones_inac45MIN_ult_20_dias,promedio_sesiones_sec_inac45MIN_ult_20_dias,tiempo_total_sesiones_sec_inac45MIN_ult_20_dias,cant_sesiones_inac45MIN_ult_15_dias,promedio_sesiones_sec_inac45MIN_ult_15_dias,tiempo_total_sesiones_sec_inac45MIN_ult_15_dias,cant_sesiones_inac45MIN_ult_10_dias,promedio_sesiones_sec_inac45MIN_ult_10_dias,tiempo_total_sesiones_sec_inac45MIN_ult_10_dias,cant_sesiones_inac45MIN_ult_5_dias,promedio_sesiones_sec_inac45MIN_ult_5_dias,tiempo_total_sesiones_sec_inac45MIN_ult_5_dias,cant_sesiones_inac45MIN_ult_1_dias,promedio_sesiones_sec_inac45MIN_ult_1_dias,tiempo_total_sesiones_sec_inac45MIN_ult_1_dias,cant_sesiones_inac30MIN_ult_9999_dias,promedio_sesiones_sec_inac30MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac30MIN_ult_9999_dias,cant_sesiones_inac30MIN_ult_30_dias,promedio_sesiones_sec_inac30MIN_ult_30_dias,tiempo_total_sesiones_sec_inac30MIN_ult_30_dias,cant_sesiones_inac30MIN_ult_25_dias,promedio_sesiones_sec_inac30MIN_ult_25_dias,tiempo_total_sesiones_sec_inac30MIN_ult_25_dias,cant_sesiones_inac30MIN_ult_20_dias,promedio_sesiones_sec_inac30MIN_ult_20_dias,tiempo_total_sesiones_sec_inac30MIN_ult_20_dias,cant_sesiones_inac30MIN_ult_15_dias,promedio_sesiones_sec_inac30MIN_ult_15_dias,tiempo_total_sesiones_sec_inac30MIN_ult_15_dias,cant_sesiones_inac30MIN_ult_10_dias,promedio_sesiones_sec_inac30MIN_ult_10_dias,tiempo_total_sesiones_sec_inac30MIN_ult_10_dias,cant_sesiones_inac30MIN_ult_5_dias,promedio_sesiones_sec_inac30MIN_ult_5_dias,tiempo_total_sesiones_sec_inac30MIN_ult_5_dias,cant_sesiones_inac30MIN_ult_1_dias,promedio_sesiones_sec_inac30MIN_ult_1_dias,tiempo_total_sesiones_sec_inac30MIN_ult_1_dias,samsung_vistos,iphone_vistos,motorola_vistos,total_marcas_vistas,%_vistos_samsung,%_vistos_motorola,%_vistos_iphone,modelos_vistos,modelos_distintos,sku_distintos,mod_mas_visto_ult_9999_dias,total_vistos_ult_9999_dias,%_mas_visto_ult_9999_dias,mod_mas_visto_ult_30_dias,total_vistos_ult_30_dias,%_mas_visto_ult_30_dias,mod_mas_visto_ult_25_dias,total_vistos_ult_25_dias,%_mas_visto_ult_25_dias,mod_mas_visto_ult_20_dias,total_vistos_ult_20_dias,%_mas_visto_ult_20_dias,mod_mas_visto_ult_15_dias,total_vistos_ult_15_dias,%_mas_visto_ult_15_dias,mod_mas_visto_ult_10_dias,total_vistos_ult_10_dias,%_mas_visto_ult_10_dias,mod_mas_visto_ult_5_dias,total_vistos_ult_5_dias,%_mas_visto_ult_5_dias,mod_mas_visto_ult_1_dias,total_vistos_ult_1_dias,%_mas_visto_ult_1_dias,#Camino,compro,#pasos_antes_comprar,primera_actividad_ultimo_paso,ultima_actividad
0,0008ed71,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0,False,6,14,14
1,00091926,31,708.0,21934.0,31.0,708.0,21934.0,27.0,761.0,20544.0,22.0,756.0,16625.0,14.0,830.0,11623.0,9.0,396.0,3562.0,5.0,244.0,1222.0,2.0,124.0,248.0,34,438.0,14890.0,34.0,438.0,14890.0,30.0,450.0,13500.0,24.0,505.0,12116.0,15.0,610.0,9149.0,9.0,396.0,3562.0,5.0,244.0,1222.0,2.0,124.0,248.0,61.0,251.0,55.0,367.0,0.166213,0.149864,0.683924,372.0,36.0,164.0,94.0,372.0,0.252688,94.0,372.0,0.252688,92.0,343.0,0.268222,92.0,307.0,0.299674,88.0,229.0,0.384279,29.0,73.0,0.397260,14.0,24.0,0.583333,6.0,10.0,0.6,0,False,448,28,0
2,00091a7a,1,347.0,347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,347.0,347.0,0.0,

In [38]:
personas = primera_actividad_feature(df, personas)

In [39]:
personas.head()

,person,cant_sesiones_inac45MIN_ult_9999_dias,promedio_sesiones_sec_inac45MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac45MIN_ult_9999_dias,cant_sesiones_inac45MIN_ult_30_dias,promedio_sesiones_sec_inac45MIN_ult_30_dias,tiempo_total_sesiones_sec_inac45MIN_ult_30_dias,cant_sesiones_inac45MIN_ult_25_dias,promedio_sesiones_sec_inac45MIN_ult_25_dias,tiempo_total_sesiones_sec_inac45MIN_ult_25_dias,cant_sesiones_inac45MIN_ult_20_dias,promedio_sesiones_sec_inac45MIN_ult_20_dias,tiempo_total_sesiones_sec_inac45MIN_ult_20_dias,cant_sesiones_inac45MIN_ult_15_dias,promedio_sesiones_sec_inac45MIN_ult_15_dias,tiempo_total_sesiones_sec_inac45MIN_ult_15_dias,cant_sesiones_inac45MIN_ult_10_dias,promedio_sesiones_sec_inac45MIN_ult_10_dias,tiempo_total_sesiones_sec_inac45MIN_ult_10_dias,cant_sesiones_inac45MIN_ult_5_dias,promedio_sesiones_sec_inac45MIN_ult_5_dias,tiempo_total_sesiones_sec_inac45MIN_ult_5_dias,cant_sesiones_inac45MIN_ult_1_dias,promedio_sesiones_sec_inac45MIN_ult_1_dias,tiempo_total_sesiones_sec_inac45MIN_ult_1_dias,cant_sesiones_inac30MIN_ult_9999_dias,promedio_sesiones_sec_inac30MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac30MIN_ult_9999_dias,cant_sesiones_inac30MIN_ult_30_dias,promedio_sesiones_sec_inac30MIN_ult_30_dias,tiempo_total_sesiones_sec_inac30MIN_ult_30_dias,cant_sesiones_inac30MIN_ult_25_dias,promedio_sesiones_sec_inac30MIN_ult_25_dias,tiempo_total_sesiones_sec_inac30MIN_ult_25_dias,cant_sesiones_inac30MIN_ult_20_dias,promedio_sesiones_sec_inac30MIN_ult_20_dias,tiempo_total_sesiones_sec_inac30MIN_ult_20_dias,cant_sesiones_inac30MIN_ult_15_dias,promedio_sesiones_sec_inac30MIN_ult_15_dias,tiempo_total_sesiones_sec_inac30MIN_ult_15_dias,cant_sesiones_inac30MIN_ult_10_dias,promedio_sesiones_sec_inac30MIN_ult_10_dias,tiempo_total_sesiones_sec_inac30MIN_ult_10_dias,cant_sesiones_inac30MIN_ult_5_dias,promedio_sesiones_sec_inac30MIN_ult_5_dias,tiempo_total_sesiones_sec_inac30MIN_ult_5_dias,cant_sesiones_inac30MIN_ult_1_dias,promedio_sesiones_sec_inac30MIN_ult_1_dias,tiempo_total_sesiones_sec_inac30MIN_ult_1_dias,samsung_vistos,iphone_vistos,motorola_vistos,total_marcas_vistas,%_vistos_samsung,%_vistos_motorola,%_vistos_iphone,modelos_vistos,modelos_distintos,sku_distintos,mod_mas_visto_ult_9999_dias,total_vistos_ult_9999_dias,%_mas_visto_ult_9999_dias,mod_mas_visto_ult_30_dias,total_vistos_ult_30_dias,%_mas_visto_ult_30_dias,mod_mas_visto_ult_25_dias,total_vistos_ult_25_dias,%_mas_visto_ult_25_dias,mod_mas_visto_ult_20_dias,total_vistos_ult_20_dias,%_mas_visto_ult_20_dias,mod_mas_visto_ult_15_dias,total_vistos_ult_15_dias,%_mas_visto_ult_15_dias,mod_mas_visto_ult_10_dias,total_vistos_ult_10_dias,%_mas_visto_ult_10_dias,mod_mas_visto_ult_5_dias,total_vistos_ult_5_dias,%_mas_visto_ult_5_dias,mod_mas_visto_ult_1_dias,total_vistos_ult_1_dias,%_mas_visto_ult_1_dias,#Camino,compro,#pasos_antes_comprar,primera_actividad_ultimo_paso,ultima_actividad,primera_actividad
0,0008ed71,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0,False,6,14,14,14
1,00091926,31,708.0,21934.0,31.0,708.0,21934.0,27.0,761.0,20544.0,22.0,756.0,16625.0,14.0,830.0,11623.0,9.0,396.0,3562.0,5.0,244.0,1222.0,2.0,124.0,248.0,34,438.0,14890.0,34.0,438.0,14890.0,30.0,450.0,13500.0,24.0,505.0,12116.0,15.0,610.0,9149.0,9.0,396.0,3562.0,5.0,244.0,1222.0,2.0,124.0,248.0,61.0,251.0,55.0,367.0,0.166213,0.149864,0.683924,372.0,36.0,164.0,94.0,372.0,0.252688,94.0,372.0,0.252688,92.0,343.0,0.268222,92.0,307.0,0.299674,88.0,229.0,0.384279,29.0,73.0,0.397260,14.0,24.0,0.583333,6.0,10.0,0.6,0,False,448,28,0,28
2,00091a7a,1,347.0,347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [40]:
personas['compro'] = personas['compro'].apply(lambda x : 1 if (x == True) else 0)

In [41]:
personas = checkout_ult_x_dias(df, personas, 9999)
personas = checkout_ult_x_dias(df, personas, 30)
personas = checkout_ult_x_dias(df, personas, 25)
personas = checkout_ult_x_dias(df, personas, 20)
personas = checkout_ult_x_dias(df, personas, 15)
personas = checkout_ult_x_dias(df, personas, 10)
personas = checkout_ult_x_dias(df, personas, 5)
personas = checkout_ult_x_dias(df, personas, 1)

In [42]:
personas.head()

,person,cant_sesiones_inac45MIN_ult_9999_dias,promedio_sesiones_sec_inac45MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac45MIN_ult_9999_dias,cant_sesiones_inac45MIN_ult_30_dias,promedio_sesiones_sec_inac45MIN_ult_30_dias,tiempo_total_sesiones_sec_inac45MIN_ult_30_dias,cant_sesiones_inac45MIN_ult_25_dias,promedio_sesiones_sec_inac45MIN_ult_25_dias,tiempo_total_sesiones_sec_inac45MIN_ult_25_dias,cant_sesiones_inac45MIN_ult_20_dias,promedio_sesiones_sec_inac45MIN_ult_20_dias,tiempo_total_sesiones_sec_inac45MIN_ult_20_dias,cant_sesiones_inac45MIN_ult_15_dias,promedio_sesiones_sec_inac45MIN_ult_15_dias,tiempo_total_sesiones_sec_inac45MIN_ult_15_dias,cant_sesiones_inac45MIN_ult_10_dias,promedio_sesiones_sec_inac45MIN_ult_10_dias,tiempo_total_sesiones_sec_inac45MIN_ult_10_dias,cant_sesiones_inac45MIN_ult_5_dias,promedio_sesiones_sec_inac45MIN_ult_5_dias,tiempo_total_sesiones_sec_inac45MIN_ult_5_dias,cant_sesiones_inac45MIN_ult_1_dias,promedio_sesiones_sec_inac45MIN_ult_1_dias,tiempo_total_sesiones_sec_inac45MIN_ult_1_dias,cant_sesiones_inac30MIN_ult_9999_dias,promedio_sesiones_sec_inac30MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac30MIN_ult_9999_dias,cant_sesiones_inac30MIN_ult_30_dias,promedio_sesiones_sec_inac30MIN_ult_30_dias,tiempo_total_sesiones_sec_inac30MIN_ult_30_dias,cant_sesiones_inac30MIN_ult_25_dias,promedio_sesiones_sec_inac30MIN_ult_25_dias,tiempo_total_sesiones_sec_inac30MIN_ult_25_dias,cant_sesiones_inac30MIN_ult_20_dias,promedio_sesiones_sec_inac30MIN_ult_20_dias,tiempo_total_sesiones_sec_inac30MIN_ult_20_dias,cant_sesiones_inac30MIN_ult_15_dias,promedio_sesiones_sec_inac30MIN_ult_15_dias,tiempo_total_sesiones_sec_inac30MIN_ult_15_dias,cant_sesiones_inac30MIN_ult_10_dias,promedio_sesiones_sec_inac30MIN_ult_10_dias,tiempo_total_sesiones_sec_inac30MIN_ult_10_dias,cant_sesiones_inac30MIN_ult_5_dias,promedio_sesiones_sec_inac30MIN_ult_5_dias,tiempo_total_sesiones_sec_inac30MIN_ult_5_dias,cant_sesiones_inac30MIN_ult_1_dias,promedio_sesiones_sec_inac30MIN_ult_1_dias,tiempo_total_sesiones_sec_inac30MIN_ult_1_dias,samsung_vistos,iphone_vistos,motorola_vistos,total_marcas_vistas,%_vistos_samsung,%_vistos_motorola,%_vistos_iphone,modelos_vistos,modelos_distintos,sku_distintos,mod_mas_visto_ult_9999_dias,total_vistos_ult_9999_dias,%_mas_visto_ult_9999_dias,mod_mas_visto_ult_30_dias,total_vistos_ult_30_dias,%_mas_visto_ult_30_dias,mod_mas_visto_ult_25_dias,total_vistos_ult_25_dias,%_mas_visto_ult_25_dias,mod_mas_visto_ult_20_dias,total_vistos_ult_20_dias,%_mas_visto_ult_20_dias,mod_mas_visto_ult_15_dias,total_vistos_ult_15_dias,%_mas_visto_ult_15_dias,mod_mas_visto_ult_10_dias,total_vistos_ult_10_dias,%_mas_visto_ult_10_dias,mod_mas_visto_ult_5_dias,total_vistos_ult_5_dias,%_mas_visto_ult_5_dias,mod_mas_visto_ult_1_dias,total_vistos_ult_1_dias,%_mas_visto_ult_1_dias,#Camino,compro,#pasos_antes_comprar,primera_actividad_ultimo_paso,ultima_actividad,primera_actividad,check_ult_9999_dias,check_ult_30_dias,check_ult_25_dias,check_ult_20_dias,check_ult_15_dias,check_ult_10_dias,check_ult_5_dias,check_ult_1_dias
0,0008ed71,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,6,14,14,14,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0
1,00091926,31,708.0,21934.0,31.0,708.0,21934.0,27.0,761.0,20544.0,22.0,756.0,16625.0,14.0,830.0,11623.0,9.0,396.0,3562.0,5.0,244.0,1222.0,2.0,124.0,248.0,34,438.0,14890.0,34.0,438.0,14890.0,30.0,450.0,13500.0,24.0,505.0,12116.0,15.0,610.0,9149.0,9.0,396.0,3562.0,5.0,244.0,1222.0,2.0,124.0,248.0,61.0,251.0,55.0,367.0,0.166213,0.149864,0.683924,372.0,36.0,164.0,94.0,372.0,0.252688,94.0,372.0,0.252688,92.0,343.0,0.268222,92.0,307.0,0.299674,88.0,229.0,0.384279,29

In [43]:
personas = ultimo_checkout(df, personas)

In [44]:
personas = mod_dist_check_ult_x_dias(df, personas, 9999)
personas = mod_dist_check_ult_x_dias(df, personas, 30)
personas = mod_dist_check_ult_x_dias(df, personas, 25)
personas = mod_dist_check_ult_x_dias(df, personas, 20)
personas = mod_dist_check_ult_x_dias(df, personas, 15)
personas = mod_dist_check_ult_x_dias(df, personas, 10)
personas = mod_dist_check_ult_x_dias(df, personas, 5)
personas = mod_dist_check_ult_x_dias(df, personas, 1)

In [45]:
personas.head()

,person,cant_sesiones_inac45MIN_ult_9999_dias,promedio_sesiones_sec_inac45MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac45MIN_ult_9999_dias,cant_sesiones_inac45MIN_ult_30_dias,promedio_sesiones_sec_inac45MIN_ult_30_dias,tiempo_total_sesiones_sec_inac45MIN_ult_30_dias,cant_sesiones_inac45MIN_ult_25_dias,promedio_sesiones_sec_inac45MIN_ult_25_dias,tiempo_total_sesiones_sec_inac45MIN_ult_25_dias,cant_sesiones_inac45MIN_ult_20_dias,promedio_sesiones_sec_inac45MIN_ult_20_dias,tiempo_total_sesiones_sec_inac45MIN_ult_20_dias,cant_sesiones_inac45MIN_ult_15_dias,promedio_sesiones_sec_inac45MIN_ult_15_dias,tiempo_total_sesiones_sec_inac45MIN_ult_15_dias,cant_sesiones_inac45MIN_ult_10_dias,promedio_sesiones_sec_inac45MIN_ult_10_dias,tiempo_total_sesiones_sec_inac45MIN_ult_10_dias,cant_sesiones_inac45MIN_ult_5_dias,promedio_sesiones_sec_inac45MIN_ult_5_dias,tiempo_total_sesiones_sec_inac45MIN_ult_5_dias,cant_sesiones_inac45MIN_ult_1_dias,promedio_sesiones_sec_inac45MIN_ult_1_dias,tiempo_total_sesiones_sec_inac45MIN_ult_1_dias,cant_sesiones_inac30MIN_ult_9999_dias,promedio_sesiones_sec_inac30MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac30MIN_ult_9999_dias,cant_sesiones_inac30MIN_ult_30_dias,promedio_sesiones_sec_inac30MIN_ult_30_dias,tiempo_total_sesiones_sec_inac30MIN_ult_30_dias,cant_sesiones_inac30MIN_ult_25_dias,promedio_sesiones_sec_inac30MIN_ult_25_dias,tiempo_total_sesiones_sec_inac30MIN_ult_25_dias,cant_sesiones_inac30MIN_ult_20_dias,promedio_sesiones_sec_inac30MIN_ult_20_dias,tiempo_total_sesiones_sec_inac30MIN_ult_20_dias,cant_sesiones_inac30MIN_ult_15_dias,promedio_sesiones_sec_inac30MIN_ult_15_dias,tiempo_total_sesiones_sec_inac30MIN_ult_15_dias,cant_sesiones_inac30MIN_ult_10_dias,promedio_sesiones_sec_inac30MIN_ult_10_dias,tiempo_total_sesiones_sec_inac30MIN_ult_10_dias,cant_sesiones_inac30MIN_ult_5_dias,promedio_sesiones_sec_inac30MIN_ult_5_dias,tiempo_total_sesiones_sec_inac30MIN_ult_5_dias,cant_sesiones_inac30MIN_ult_1_dias,promedio_sesiones_sec_inac30MIN_ult_1_dias,tiempo_total_sesiones_sec_inac30MIN_ult_1_dias,samsung_vistos,iphone_vistos,motorola_vistos,total_marcas_vistas,%_vistos_samsung,%_vistos_motorola,%_vistos_iphone,modelos_vistos,modelos_distintos,sku_distintos,mod_mas_visto_ult_9999_dias,total_vistos_ult_9999_dias,%_mas_visto_ult_9999_dias,mod_mas_visto_ult_30_dias,total_vistos_ult_30_dias,%_mas_visto_ult_30_dias,mod_mas_visto_ult_25_dias,total_vistos_ult_25_dias,%_mas_visto_ult_25_dias,mod_mas_visto_ult_20_dias,total_vistos_ult_20_dias,%_mas_visto_ult_20_dias,mod_mas_visto_ult_15_dias,total_vistos_ult_15_dias,%_mas_visto_ult_15_dias,mod_mas_visto_ult_10_dias,total_vistos_ult_10_dias,%_mas_visto_ult_10_dias,mod_mas_visto_ult_5_dias,total_vistos_ult_5_dias,%_mas_visto_ult_5_dias,mod_mas_visto_ult_1_dias,total_vistos_ult_1_dias,%_mas_visto_ult_1_dias,#Camino,compro,#pasos_antes_comprar,primera_actividad_ultimo_paso,ultima_actividad,primera_actividad,check_ult_9999_dias,check_ult_30_dias,check_ult_25_dias,check_ult_20_dias,check_ult_15_dias,check_ult_10_dias,check_ult_5_dias,check_ult_1_dias,ult_checkout_dias,ult_checkout_segundos,mod_dist_check_ult_9999_dias,mod_dist_check_ult_30_dias,mod_dist_check_ult_25_dias,mod_dist_check_ult_20_dias,mod_dist_check_ult_15_dias,mod_dist_check_ult_10_dias,mod_dist_check_ult_5_dias,mod_dist_check_ult_1_dias
0,0008ed71,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,6,14,14,14,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,14.0,1236682.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0
1,00091926,31,708.0,21934.0,31.0,708.0,21934.0,27.0,761.0,20544.0,22.0,756.0,16625.0,14.0,830.0,11623.0,9.0,396.0,3562.0,5.0,244.0,1222.0,2.0,124.0,248.0,34,438.

In [46]:
personas = actividad_ult_x_dias(df, personas, 9999)
personas = actividad_ult_x_dias(df, personas, 30)
personas = actividad_ult_x_dias(df, personas, 25)
personas = actividad_ult_x_dias(df, personas, 20)
personas = actividad_ult_x_dias(df, personas, 15)
personas = actividad_ult_x_dias(df, personas, 10)
personas = actividad_ult_x_dias(df, personas, 5)
personas = actividad_ult_x_dias(df, personas, 1)

In [47]:
personas.head()

,person,cant_sesiones_inac45MIN_ult_9999_dias,promedio_sesiones_sec_inac45MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac45MIN_ult_9999_dias,cant_sesiones_inac45MIN_ult_30_dias,promedio_sesiones_sec_inac45MIN_ult_30_dias,tiempo_total_sesiones_sec_inac45MIN_ult_30_dias,cant_sesiones_inac45MIN_ult_25_dias,promedio_sesiones_sec_inac45MIN_ult_25_dias,tiempo_total_sesiones_sec_inac45MIN_ult_25_dias,cant_sesiones_inac45MIN_ult_20_dias,promedio_sesiones_sec_inac45MIN_ult_20_dias,tiempo_total_sesiones_sec_inac45MIN_ult_20_dias,cant_sesiones_inac45MIN_ult_15_dias,promedio_sesiones_sec_inac45MIN_ult_15_dias,tiempo_total_sesiones_sec_inac45MIN_ult_15_dias,cant_sesiones_inac45MIN_ult_10_dias,promedio_sesiones_sec_inac45MIN_ult_10_dias,tiempo_total_sesiones_sec_inac45MIN_ult_10_dias,cant_sesiones_inac45MIN_ult_5_dias,promedio_sesiones_sec_inac45MIN_ult_5_dias,tiempo_total_sesiones_sec_inac45MIN_ult_5_dias,cant_sesiones_inac45MIN_ult_1_dias,promedio_sesiones_sec_inac45MIN_ult_1_dias,tiempo_total_sesiones_sec_inac45MIN_ult_1_dias,cant_sesiones_inac30MIN_ult_9999_dias,promedio_sesiones_sec_inac30MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac30MIN_ult_9999_dias,cant_sesiones_inac30MIN_ult_30_dias,promedio_sesiones_sec_inac30MIN_ult_30_dias,tiempo_total_sesiones_sec_inac30MIN_ult_30_dias,cant_sesiones_inac30MIN_ult_25_dias,promedio_sesiones_sec_inac30MIN_ult_25_dias,tiempo_total_sesiones_sec_inac30MIN_ult_25_dias,cant_sesiones_inac30MIN_ult_20_dias,promedio_sesiones_sec_inac30MIN_ult_20_dias,tiempo_total_sesiones_sec_inac30MIN_ult_20_dias,cant_sesiones_inac30MIN_ult_15_dias,promedio_sesiones_sec_inac30MIN_ult_15_dias,tiempo_total_sesiones_sec_inac30MIN_ult_15_dias,cant_sesiones_inac30MIN_ult_10_dias,promedio_sesiones_sec_inac30MIN_ult_10_dias,tiempo_total_sesiones_sec_inac30MIN_ult_10_dias,cant_sesiones_inac30MIN_ult_5_dias,promedio_sesiones_sec_inac30MIN_ult_5_dias,tiempo_total_sesiones_sec_inac30MIN_ult_5_dias,cant_sesiones_inac30MIN_ult_1_dias,promedio_sesiones_sec_inac30MIN_ult_1_dias,tiempo_total_sesiones_sec_inac30MIN_ult_1_dias,samsung_vistos,iphone_vistos,motorola_vistos,total_marcas_vistas,%_vistos_samsung,%_vistos_motorola,%_vistos_iphone,modelos_vistos,modelos_distintos,sku_distintos,mod_mas_visto_ult_9999_dias,total_vistos_ult_9999_dias,%_mas_visto_ult_9999_dias,mod_mas_visto_ult_30_dias,total_vistos_ult_30_dias,%_mas_visto_ult_30_dias,mod_mas_visto_ult_25_dias,total_vistos_ult_25_dias,%_mas_visto_ult_25_dias,mod_mas_visto_ult_20_dias,total_vistos_ult_20_dias,%_mas_visto_ult_20_dias,mod_mas_visto_ult_15_dias,total_vistos_ult_15_dias,%_mas_visto_ult_15_dias,mod_mas_visto_ult_10_dias,total_vistos_ult_10_dias,%_mas_visto_ult_10_dias,mod_mas_visto_ult_5_dias,total_vistos_ult_5_dias,%_mas_visto_ult_5_dias,mod_mas_visto_ult_1_dias,total_vistos_ult_1_dias,%_mas_visto_ult_1_dias,#Camino,compro,#pasos_antes_comprar,primera_actividad_ultimo_paso,ultima_actividad,primera_actividad,check_ult_9999_dias,check_ult_30_dias,check_ult_25_dias,check_ult_20_dias,check_ult_15_dias,check_ult_10_dias,check_ult_5_dias,check_ult_1_dias,ult_checkout_dias,ult_checkout_segundos,mod_dist_check_ult_9999_dias,mod_dist_check_ult_30_dias,mod_dist_check_ult_25_dias,mod_dist_check_ult_20_dias,mod_dist_check_ult_15_dias,mod_dist_check_ult_10_dias,mod_dist_check_ult_5_dias,mod_dist_check_ult_1_dias,act_ult_9999_dias,act_ult_30_dias,act_ult_25_dias,act_ult_20_dias,act_ult_15_dias,act_ult_10_dias,act_ult_5_dias,act_ult_1_dias
0,0008ed71,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0,0,6,14,14,14,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,14.0,1236682.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,6,6.0,6.0,6.0,6.0,0.0,0.0,0.0
1,0

In [48]:
personas = conv_ult_x_dias(df, personas, 9999)
personas = conv_ult_x_dias(df, personas, 30)
personas = conv_ult_x_dias(df, personas, 25)
personas = conv_ult_x_dias(df, personas, 20)
personas = conv_ult_x_dias(df, personas, 15)
personas = conv_ult_x_dias(df, personas, 10)
personas = conv_ult_x_dias(df, personas, 5)
personas = conv_ult_x_dias(df, personas, 1)

In [49]:
personas.head()

,person,cant_sesiones_inac45MIN_ult_9999_dias,promedio_sesiones_sec_inac45MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac45MIN_ult_9999_dias,cant_sesiones_inac45MIN_ult_30_dias,promedio_sesiones_sec_inac45MIN_ult_30_dias,tiempo_total_sesiones_sec_inac45MIN_ult_30_dias,cant_sesiones_inac45MIN_ult_25_dias,promedio_sesiones_sec_inac45MIN_ult_25_dias,tiempo_total_sesiones_sec_inac45MIN_ult_25_dias,cant_sesiones_inac45MIN_ult_20_dias,promedio_sesiones_sec_inac45MIN_ult_20_dias,tiempo_total_sesiones_sec_inac45MIN_ult_20_dias,cant_sesiones_inac45MIN_ult_15_dias,promedio_sesiones_sec_inac45MIN_ult_15_dias,tiempo_total_sesiones_sec_inac45MIN_ult_15_dias,cant_sesiones_inac45MIN_ult_10_dias,promedio_sesiones_sec_inac45MIN_ult_10_dias,tiempo_total_sesiones_sec_inac45MIN_ult_10_dias,cant_sesiones_inac45MIN_ult_5_dias,promedio_sesiones_sec_inac45MIN_ult_5_dias,tiempo_total_sesiones_sec_inac45MIN_ult_5_dias,cant_sesiones_inac45MIN_ult_1_dias,promedio_sesiones_sec_inac45MIN_ult_1_dias,tiempo_total_sesiones_sec_inac45MIN_ult_1_dias,cant_sesiones_inac30MIN_ult_9999_dias,promedio_sesiones_sec_inac30MIN_ult_9999_dias,tiempo_total_sesiones_sec_inac30MIN_ult_9999_dias,cant_sesiones_inac30MIN_ult_30_dias,promedio_sesiones_sec_inac30MIN_ult_30_dias,tiempo_total_sesiones_sec_inac30MIN_ult_30_dias,cant_sesiones_inac30MIN_ult_25_dias,promedio_sesiones_sec_inac30MIN_ult_25_dias,tiempo_total_sesiones_sec_inac30MIN_ult_25_dias,cant_sesiones_inac30MIN_ult_20_dias,promedio_sesiones_sec_inac30MIN_ult_20_dias,tiempo_total_sesiones_sec_inac30MIN_ult_20_dias,cant_sesiones_inac30MIN_ult_15_dias,promedio_sesiones_sec_inac30MIN_ult_15_dias,tiempo_total_sesiones_sec_inac30MIN_ult_15_dias,cant_sesiones_inac30MIN_ult_10_dias,promedio_sesiones_sec_inac30MIN_ult_10_dias,tiempo_total_sesiones_sec_inac30MIN_ult_10_dias,cant_sesiones_inac30MIN_ult_5_dias,promedio_sesiones_sec_inac30MIN_ult_5_dias,tiempo_total_sesiones_sec_inac30MIN_ult_5_dias,cant_sesiones_inac30MIN_ult_1_dias,promedio_sesiones_sec_inac30MIN_ult_1_dias,tiempo_total_sesiones_sec_inac30MIN_ult_1_dias,samsung_vistos,iphone_vistos,motorola_vistos,total_marcas_vistas,%_vistos_samsung,%_vistos_motorola,%_vistos_iphone,modelos_vistos,modelos_distintos,sku_distintos,mod_mas_visto_ult_9999_dias,total_vistos_ult_9999_dias,%_mas_visto_ult_9999_dias,mod_mas_visto_ult_30_dias,total_vistos_ult_30_dias,%_mas_visto_ult_30_dias,mod_mas_visto_ult_25_dias,total_vistos_ult_25_dias,%_mas_visto_ult_25_dias,mod_mas_visto_ult_20_dias,total_vistos_ult_20_dias,%_mas_visto_ult_20_dias,mod_mas_visto_ult_15_dias,total_vistos_ult_15_dias,%_mas_visto_ult_15_dias,mod_mas_visto_ult_10_dias,total_vistos_ult_10_dias,%_mas_visto_ult_10_dias,mod_mas_visto_ult_5_dias,total_vistos_ult_5_dias,%_mas_visto_ult_5_dias,mod_mas_visto_ult_1_dias,total_vistos_ult_1_dias,%_mas_visto_ult_1_dias,#Camino,compro,#pasos_antes_comprar,primera_actividad_ultimo_paso,ultima_actividad,primera_actividad,check_ult_9999_dias,check_ult_30_dias,check_ult_25_dias,check_ult_20_dias,check_ult_15_dias,check_ult_10_dias,check_ult_5_dias,check_ult_1_dias,ult_checkout_dias,ult_checkout_segundos,mod_dist_check_ult_9999_dias,mod_dist_check_ult_30_dias,mod_dist_check_ult_25_dias,mod_dist_check_ult_20_dias,mod_dist_check_ult_15_dias,mod_dist_check_ult_10_dias,mod_dist_check_ult_5_dias,mod_dist_check_ult_1_dias,act_ult_9999_dias,act_ult_30_dias,act_ult_25_dias,act_ult_20_dias,act_ult_15_dias,act_ult_10_dias,act_ult_5_dias,act_ult_1_dias,conv_ult_9999_dias,conv_ult_30_dias,conv_ult_25_dias,conv_ult_20_dias,conv_ult_15_dias,conv_ult_10_dias,conv_ult_5_dias,conv_ult_1_dias
0,0008ed71,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,3.0,135.0,404.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0

In [50]:
#personas = info_demografica(df, personas)

In [292]:
personas.to_csv("personas_features.csv", index = False)

In [295]:
trainingData = pd.read_csv(trainingFile)
training = personas.merge(trainingData, on = 'person', how = 'left', indicator = True)


In [296]:
training.to_csv("features_Raul.csv", index = False)